# Traitement des données

In [13]:
import pyspark

In [14]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Traitement") \
    .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/SpankFoot.ucl_clubs_attack") \
    .config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/SpankFoot.ucl_clubs_attack") \
    .getOrCreate()

In [744]:
spark = SparkSession.builder.appName("Traitement") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/SpankFoot.ucl_clubs_attack") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/SpankFoot.ucl_clubs_attack") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.3.1') \
    .getOrCreate()

In [149]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import split
import pandas as pd 
from pyspark.sql.functions import lit

## UCL

### Attack
 

In [19]:
path = "./data/LDC/ucl_clubs_attack.csv"
df1 = spark.read.option("delimiter",",").options(header=True).csv(path)
df1.dtypes

[('_c0', 'string'), ('team', 'string')]

In [20]:
df1.show()

+---+--------------------+
|_c0|                team|
+---+--------------------+
|  0|01 Man_City Engla...|
|  1|02 Barcelona Spai...|
|  2|03 Real_Madrid Sp...|
|  3|04 Liverpool Engl...|
|  4|05 Marseille Fran...|
|  5|06 Chelsea Englan...|
|  6|07 Paris France 3...|
|  7|08 Atlético Spain...|
|  8|09 Bayern Germany...|
|  9|10 Juventus Italy...|
| 10|11 Leverkusen Ger...|
| 11|12 Benfica Portug...|
| 12|13 Napoli Italy 2...|
| 13|14 Milan Italy 25...|
| 14|15 Salzburg Austr...|
+---+--------------------+



In [21]:
df1 = df1.drop(col("_c0"))
df1 = df1.withColumn('Rank', split(df1['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df1['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df1['team'], ' ').getItem(2)) \
       .withColumn('Attack', split(df1['team'], ' ').getItem(3)) \
       .withColumn('Assist', split(df1['team'], ' ').getItem(4)) \
       .withColumn('Corner_taken', split(df1['team'], ' ').getItem(5)) \
       .withColumn('Offside', split(df1['team'], ' ').getItem(8)) \
       .withColumn('Dribble', split(df1['team'], ' ').getItem(6)) \
       .withColumn('Match_played', split(df1['team'], ' ').getItem(7))

df1 = df1.drop(col("team"))
df1 = df1.na.drop()
df1.show(truncate=False)

+----+-----------+--------+------+------+------------+-------+-------+------------+
|Rank|Equipe     |Country |Attack|Assist|Corner_taken|Offside|Dribble|Match_played|
+----+-----------+--------+------+------+------------+-------+-------+------------+
|01  |Man_City   |England |415   |12    |38          |6      |8      |82          |
|02  |Barcelona  |Spain   |398   |11    |40          |6      |8      |107         |
|03  |Real_Madrid|Spain   |372   |11    |40          |6      |6      |99          |
|04  |Liverpool  |England |349   |14    |46          |6      |12     |83          |
|05  |Marseille  |France  |336   |6     |30          |6      |26     |89          |
|06  |Chelsea    |England |333   |7     |27          |6      |16     |73          |
|07  |Paris      |France  |317   |13    |29          |6      |14     |127         |
|08  |Atlético   |Spain   |309   |5     |39          |6      |20     |83          |
|09  |Bayern     |Germany |305   |17    |19          |6      |17     |91    

In [22]:
df1.toPandas().to_csv("./data/LDC/ucl_clubs_attack.csv")

### Games 

In [23]:
path = "./data/LDC/ucl_games.csv"
df2 = spark.read.option("delimiter",",").options(header=True).csv(path)
df2 = df2.drop(col("_c0"))

In [24]:
df2.show()

+------------+-------------------+---------+---------+-------------------+
|        date|         equipe_dom|score_dom|score_ext|         equipe_ext|
+------------+-------------------+---------+---------+-------------------+
|02.11. 21:00|           AC Milan|        4|        0|           Salzburg|
|02.11. 21:00|            Chelsea|        2|        1|          D. Zagreb|
|02.11. 21:00|      FC Copenhagen|        1|        1|           Dortmund|
|02.11. 21:00|           Juventus|        1|        2|           Paris SG|
|02.11. 21:00|      Maccabi Haifa|        1|        6|            Benfica|
|02.11. 21:00|    Manchester City|        3|        1|            Sevilla|
|02.11. 18:45|        Real Madrid|        5|        1|             Celtic|
|02.11. 18:45|   Shakhtar Donetsk|        0|        4|         RB Leipzig|
|01.11. 21:00|      Bayern Munich|        2|        0|              Inter|
|01.11. 21:00|          Liverpool|        2|        0|             Napoli|
|01.11. 21:00|          M

In [25]:
#df2.write.option("header",True).csv("./data/LDC/ucl_games")
df2.toPandas().to_csv("./data/LDC/ucl_games.csv")

### Group

In [173]:
path = "./data/LDC/ucl_group.csv"
df3 = spark.read.option("delimiter",",").options(header=True).csv(path)
df3 = df3.drop(col("_c0"))
df3.show(50)

+----------------+--------+---+----+----+---+---+---+---+
|            team|nb_match|win|draw|lose| GF| GA| GD|pts|
+----------------+--------+---+----+----+---+---+---+---+
|          Napoli|       6|  5|   0|   1| 20|  6| 14| 15|
|       Liverpool|       6|  5|   0|   1| 17|  6| 11| 15|
|            Ajax|       6|  2|   0|   4| 11| 16| -5|  6|
|         Rangers|       6|  0|   0|   6|  2| 22|-20|  0|
|           Porto|       6|  4|   0|   2| 12|  7|  5| 12|
|     Club Brugge|       6|  3|   2|   1|  7|  4|  3| 11|
|      Leverkusen|       6|  1|   2|   3|  4|  8| -4|  5|
|        Atlético|       6|  1|   2|   3|  5|  9| -4|  5|
|          Bayern|       6|  6|   0|   0| 18|  2| 16| 18|
|           Inter|       6|  3|   1|   2| 10|  7|  3| 10|
|       Barcelona|       6|  2|   1|   3| 12| 12|  0|  7|
|           Plzeň|       6|  0|   0|   6|  5| 24|-19|  0|
|       Tottenham|       6|  3|   2|   1|  8|  6|  2| 11|
|       Frankfurt|       6|  3|   1|   2|  7|  8| -1| 10|
|     Sporting

In [180]:
from pyspark.sql.functions import when
df3 = df3.withColumn("Groupe", \
   when((df3.team == "Napoli"), lit("A")) \
     .when((df3.team == "Liverpool"), lit("A")) \
     .when((df3.team == "Ajax") , lit("A")) \
     .when((df3.team == "Rangers") , lit("A")) \
     .when((df3.team == "Porto"), lit("B")) \
     .when((df3.team == "Club Brugge") , lit("B")) \
     .when((df3.team == "Leverkusen") , lit("B")) \
     .when((df3.team == "Atlético") , lit("B")) \
     .when((df3.team == "Bayern"), lit("C")) \
     .when((df3.team == "Inter") , lit("C")) \
     .when((df3.team == "Barcelona") , lit("C")) \
     .when((df3.team == "Plzeň") , lit("C")) \
     .when((df3.team == "Tottenham") , lit("D")) \
     .when((df3.team == "Frankfurt") , lit("D")) \
     .when((df3.team == "Sporting CP") , lit("D")) \
     .when((df3.team == "Marseille") , lit("D")) \
     .when((df3.team == "Chelsea") , lit("E")) \
     .when((df3.team == "Milan") , lit("E")) \
     .when((df3.team == "Salzburg") , lit("E")) \
     .when((df3.team == "Dinamo Zagreb") , lit("E")) \
     .when((df3.team == "Real Madrid") , lit("F")) \
     .when((df3.team == "Leipzig") , lit("F")) \
     .when((df3.team == "Shakhtar Donetsk") , lit("F")) \
     .when((df3.team == "Celtic") , lit("F")) \
     .when((df3.team == "Man City") , lit("G")) \
     .when((df3.team == "Dortmund") , lit("G")) \
     .when((df3.team == "Sevilla") , lit("G")) \
     .when((df3.team == "Copenhagen") , lit("G")) \
     .when((df3.team == "Benfica") , lit("H")) \
     .when((df3.team == "Paris") , lit("H")) \
     .when((df3.team == "Juventus") , lit("H")) \
     .when((df3.team == "M. Haifa" ) , lit("H")))
df3.show(32)     

+----------------+--------+---+----+----+---+---+---+---+------+
|            team|nb_match|win|draw|lose| GF| GA| GD|pts|Groupe|
+----------------+--------+---+----+----+---+---+---+---+------+
|          Napoli|       6|  5|   0|   1| 20|  6| 14| 15|     A|
|       Liverpool|       6|  5|   0|   1| 17|  6| 11| 15|     A|
|            Ajax|       6|  2|   0|   4| 11| 16| -5|  6|     A|
|         Rangers|       6|  0|   0|   6|  2| 22|-20|  0|     A|
|           Porto|       6|  4|   0|   2| 12|  7|  5| 12|     B|
|     Club Brugge|       6|  3|   2|   1|  7|  4|  3| 11|     B|
|      Leverkusen|       6|  1|   2|   3|  4|  8| -4|  5|     B|
|        Atlético|       6|  1|   2|   3|  5|  9| -4|  5|     B|
|          Bayern|       6|  6|   0|   0| 18|  2| 16| 18|     C|
|           Inter|       6|  3|   1|   2| 10|  7|  3| 10|     C|
|       Barcelona|       6|  2|   1|   3| 12| 12|  0|  7|     C|
|           Plzeň|       6|  0|   0|   6|  5| 24|-19|  0|     C|
|       Tottenham|       

In [181]:
df3.toPandas().to_csv("./data/LDC/ucl_group.csv")

### DEF

In [28]:
path = "./data/LDC/ucl_clubs_def.csv"
df4 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [29]:
df4 = df4.drop(col("_c0"))
df4 = df4.withColumn('Rank', split(df4['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df4['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df4['team'], ' ').getItem(2)) \
       .withColumn('Ball_recover', split(df4['team'], ' ').getItem(3)) \
       .withColumn('Tackles', split(df4['team'], ' ').getItem(4)) \
       .withColumn('Tackles_Won', split(df4['team'], ' ').getItem(5)) \
       .withColumn('Tackles_Lost', split(df4['team'], ' ').getItem(8)) \
       .withColumn('Clearence_attempt', split(df4['team'], ' ').getItem(6)) \
       .withColumn('Match_played', split(df4['team'], ' ').getItem(7))

df4 = df4.drop(col("team"))
df4 = df4.na.drop()
df4.show(truncate=False)

+----+----------+-----------+------------+-------+-----------+------------+-----------------+------------+
|Rank|Equipe    |Country    |Ball_recover|Tackles|Tackles_Won|Tackles_Lost|Clearence_attempt|Match_played|
+----+----------+-----------+------------+-------+-----------+------------+-----------------+------------+
|01  |Frankfurt |Germany    |296         |67     |23         |44          |88               |6           |
|02  |Chelsea   |England    |278         |66     |25         |41          |82               |6           |
|03  |Salzburg  |Austria    |272         |101    |46         |55          |125              |6           |
|04  |Leverkusen|Germany    |266         |78     |27         |51          |127              |6           |
|05  |Bayern    |Germany    |255         |85     |32         |53          |78               |6           |
|05  |Napoli    |Italy      |255         |66     |25         |41          |112              |6           |
|07  |Benfica   |Portugal   |252     

In [30]:
df4.toPandas().to_csv("./data/LDC/ucl_clubs_def.csv")

### GK

In [31]:
path = "./data/LDC/ucl_clubs_gk.csv"
df5 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [32]:
df5 = df5.drop(col("_c0"))
df5 = df5.withColumn('Rank', split(df5['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df5['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df5['team'], ' ').getItem(2)) \
       .withColumn('Save', split(df5['team'], ' ').getItem(3)) \
       .withColumn('Goal_conceded', split(df5['team'], ' ').getItem(4)) \
       .withColumn('Own_goals_conceded', split(df5['team'], ' ').getItem(5)) \
       .withColumn('Saves_from_penalty', split(df5['team'], ' ').getItem(6)) \
       .withColumn('Punches_made', split(df5['team'], ' ').getItem(7)) \
       .withColumn('Match_played', split(df5['team'], ' ').getItem(8))

df5 = df5.drop(col("team"))
df5 = df5.na.drop()
df5.show(truncate=False)

+----+----------------+-----------+--------+-------------+------------------+------------------+------------+------------+
|Rank|Equipe          |Country    |Save    |Goal_conceded|Own_goals_conceded|Saves_from_penalty|Punches_made|Match_played|
+----+----------------+-----------+--------+-------------+------------------+------------------+------------+------------+
|01  |Plzeň           |Czech      |Republic|32           |24                |0                 |0           |2           |
|02  |Shakhtar_Donetsk|Ukraine    |31      |10           |2                 |0                 |10          |6           |
|02  |Club_Brugge     |Belgium    |31      |4            |0                 |5                 |5           |6           |
|04  |Rangers         |Scotland   |29      |22           |0                 |0                 |0           |6           |
|04  |Inter           |Italy      |29      |7            |1                 |3                 |4           |6           |
|06  |Copenhagen

In [33]:
df5.toPandas().to_csv("./data/LDC/ucl_clubs_gk.csv")

### Attempt

In [34]:
path = "./data/LDC/ucl_clubs_attempt.csv"
df6 = spark.read.option("delimiter",",").options(header=True).csv(path)


In [35]:
df6 = df6.drop(col("_c0"))
df6 = df6.withColumn('Rank', split(df6['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df6['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df6['team'], ' ').getItem(2)) \
       .withColumn('Total_attempt', split(df6['team'], ' ').getItem(3)) \
       .withColumn('Attempt_on_target', split(df6['team'], ' ').getItem(4)) \
       .withColumn('Attempt_off_target', split(df6['team'], ' ').getItem(5)) \
       .withColumn('Blocked', split(df6['team'], ' ').getItem(7)) \
       .withColumn('Match_played', split(df6['team'], ' ').getItem(6)) 

df6 = df6.drop(col("team"))
df6 = df6.na.drop()
df6.show(truncate=False)

+----+-----------+--------+-------------+-----------------+------------------+-------+------------+
|Rank|Equipe     |Country |Total_attempt|Attempt_on_target|Attempt_off_target|Blocked|Match_played|
+----+-----------+--------+-------------+-----------------+------------------+-------+------------+
|01  |Real_Madrid|Spain   |114          |42               |49                |23     |6           |
|02  |Liverpool  |England |110          |47               |39                |24     |6           |
|03  |Man_City   |England |109          |42               |36                |31     |6           |
|04  |Bayern     |Germany |106          |51               |34                |21     |6           |
|04  |Napoli     |Italy   |106          |44               |43                |19     |6           |
|06  |Atlético   |Spain   |94           |35               |33                |26     |6           |
|07  |Barcelona  |Spain   |92           |34               |33                |25     |6           |


In [36]:
df6.toPandas().to_csv("./data/LDC/ucl_clubs_attempt.csv")

### Goals

In [37]:
path = "./data/LDC/ucl_clubs_goals.csv"
df7 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [38]:
df7 = df7.drop(col("_c0"))
df7 = df7.withColumn('Rank', split(df7['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df7['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df7['team'], ' ').getItem(2)) \
       .withColumn('Goals', split(df7['team'], ' ').getItem(3)) \
       .withColumn('Right_foot', split(df7['team'], ' ').getItem(4)) \
       .withColumn('Left_foot', split(df7['team'], ' ').getItem(5)) \
       .withColumn('Head', split(df7['team'], ' ').getItem(11)) \
       .withColumn('Others', split(df7['team'], ' ').getItem(6)) \
       .withColumn('Goals_inside_area', split(df7['team'], ' ').getItem(7)) \
       .withColumn('Goal_outside', split(df7['team'], ' ').getItem(8)) \
       .withColumn('Penalty', split(df7['team'], ' ').getItem(9)) \
       .withColumn('Match_Played', split(df7['team'], ' ').getItem(10)) 

df7= df7.drop(col("team"))
df7 = df7.na.drop()
df7.show(truncate=False)

+----+----------------+-----------+--------+----------+---------+----+------+-----------------+------------+-------+------------+
|Rank|Equipe          |Country    |Goals   |Right_foot|Left_foot|Head|Others|Goals_inside_area|Goal_outside|Penalty|Match_Played|
+----+----------------+-----------+--------+----------+---------+----+------+-----------------+------------+-------+------------+
|01  |Napoli          |Italy      |20      |8         |7        |5   |0     |19               |1           |3      |6           |
|02  |Bayern          |Germany    |18      |12        |3        |2   |0     |15               |2           |0      |6           |
|03  |Liverpool       |England    |17      |5         |9        |3   |0     |15               |2           |1      |6           |
|04  |Paris           |France     |16      |8         |7        |0   |0     |11               |4           |1      |6           |
|04  |Benfica         |Portugal   |16      |7         |4        |3   |0     |12           

In [39]:
df7.toPandas().to_csv("./data/LDC/ucl_clubs_goals.csv")

### Key

In [40]:
path = "./data/LDC/ucl_clubs_key.csv"
df8 = spark.read.option("delimiter",",").options(header=True).csv(path)


In [41]:
df8 = df8.drop(col("_c0"))
df8 = df8.withColumn('Rank', split(df8['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df8['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df8['team'], ' ').getItem(2)) \
       .withColumn('Match_Played', split(df8['team'], ' ').getItem(3)) \
       .withColumn('Won', split(df8['team'], ' ').getItem(4)) \
       .withColumn('Drawn', split(df8['team'], ' ').getItem(5)) \
       .withColumn('Lost', split(df8['team'], ' ').getItem(6)) 

df8 = df8.drop(col("team"))
df8 = df8.na.drop()
df8.show(truncate=False)

+----+-----------+--------+------------+---+-----+----+
|Rank|Equipe     |Country |Match_Played|Won|Drawn|Lost|
+----+-----------+--------+------------+---+-----+----+
|01  |Bayern     |Germany |6           |6  |0    |0   |
|01  |Liverpool  |England |6           |5  |0    |1   |
|01  |Napoli     |Italy   |6           |5  |0    |1   |
|01  |Man_City   |England |6           |4  |2    |0   |
|01  |Paris      |France  |6           |4  |2    |0   |
|01  |Benfica    |Portugal|6           |4  |2    |0   |
|01  |Chelsea    |England |6           |4  |1    |1   |
|01  |Real_Madrid|Spain   |6           |4  |1    |1   |
|01  |Porto      |Portugal|6           |4  |0    |2   |
|01  |Leipzig    |Germany |6           |4  |0    |2   |
|01  |Club_Brugge|Belgium |6           |3  |2    |1   |
|01  |Tottenham  |England |6           |3  |2    |1   |
|01  |Inter      |Italy   |6           |3  |1    |2   |
|01  |Frankfurt  |Germany |6           |3  |1    |2   |
|01  |Milan      |Italy   |6           |3  |1   

In [42]:
df8.toPandas().to_csv("./data/LDC/ucl_clubs_key.csv")

### Discip 

In [43]:
path = "./data/LDC/ucl_clubs_discip.csv"
df9 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [44]:
df9 = df9.drop(col("_c0"))
df9 = df9.withColumn('Rank', split(df9['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df9['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df9['team'], ' ').getItem(2)) \
       .withColumn('Fouls_Commited', split(df9['team'], ' ').getItem(3)) \
       .withColumn('Fouls_Suffered', split(df9['team'], ' ').getItem(4)) \
       .withColumn('Yellow_card', split(df9['team'], ' ').getItem(5)) \
       .withColumn('Red_card', split(df9['team'], ' ').getItem(7)) \
       .withColumn('Match_Played', split(df9['team'], ' ').getItem(6)) 


df9 = df9.drop(col("team"))
df9 = df9.na.drop()
df9.show(truncate=False)

+----+----------+-----------+--------------+--------------+-----------+--------+------------+
|Rank|Equipe    |Country    |Fouls_Commited|Fouls_Suffered|Yellow_card|Red_card|Match_Played|
+----+----------+-----------+--------------+--------------+-----------+--------+------------+
|01  |Chelsea   |England    |82            |83            |12         |0       |6           |
|01  |Frankfurt |Germany    |82            |63            |14         |1       |6           |
|03  |Leverkusen|Germany    |80            |50            |19         |1       |6           |
|04  |Ajax      |Netherlands|78            |48            |15         |1       |6           |
|05  |Salzburg  |Austria    |77            |50            |11         |0       |6           |
|05  |M._Haifa  |Israel     |77            |46            |14         |0       |6           |
|07  |Milan     |Italy      |74            |62            |15         |1       |6           |
|07  |Bayern    |Germany    |74            |61            |1

In [45]:
df9.toPandas().to_csv("./data/LDC/ucl_clubs_discip.csv")

### Distrib

In [46]:
path = "./data/LDC/ucl_clubs_distrib.csv"
df10 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [47]:
df10 = df10.drop(col("_c0"))
df10 = df10.withColumn('Rank', split(df10['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df10['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df10['team'], ' ').getItem(2)) \
       .withColumn('Pass_accuracy', split(df10['team'], ' ').getItem(3)) \
       .withColumn('Pass_attempted', split(df10['team'], ' ').getItem(4)) \
       .withColumn('Pass_completed', split(df10['team'], ' ').getItem(5)) \
       .withColumn('Cross_accuracy', split(df10['team'], ' ').getItem(6)) \
       .withColumn('Cross_attempted', split(df10['team'], ' ').getItem(7)) \
       .withColumn('Cross_completed', split(df10['team'], ' ').getItem(8)) \
       .withColumn('Freekick', split(df10['team'], ' ').getItem(9)) \
       .withColumn('Match_Played', split(df10['team'], ' ').getItem(10)) \
       .withColumn('Possesion', split(df10['team'], ' ').getItem(11)) 


df10 = df10.drop(col("team"))
df10 = df10.na.drop()
df10.show(truncate=False)

+----+----------------+-----------+-------------+--------------+--------------+--------------+---------------+---------------+--------+------------+---------+
|Rank|Equipe          |Country    |Pass_accuracy|Pass_attempted|Pass_completed|Cross_accuracy|Cross_attempted|Cross_completed|Freekick|Match_Played|Possesion|
+----+----------------+-----------+-------------+--------------+--------------+--------------+---------------+---------------+--------+------------+---------+
|01  |Real_Madrid     |Spain      |91.8         |4121          |3782          |23            |105            |25             |60      |6           |57.8     |
|02  |Man_City        |England    |91.5         |4403          |4030          |23            |132            |31             |51      |6           |64.2     |
|03  |Paris           |France     |90.5         |3821          |3466          |19            |68             |13             |109     |6           |55.5     |
|04  |Barcelona       |Spain      |88.8       

In [48]:
df10.toPandas().to_csv("./data/LDC/ucl_clubs_distrib.csv")

### Players GK 

In [49]:
path = "./data/LDC/ucl_players_gk.csv"
df11 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [50]:
df11 = df11.drop(col("_c0"))
df11 = df11.withColumn('Rank', split(df11['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df11['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df11['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df11['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df11['team'], ' ').getItem(4)) \
       .withColumn('Save', split(df11['team'], ' ').getItem(5)) \
       .withColumn('Goal_conceded', split(df11['team'], ' ').getItem(6)) \
       .withColumn('Saves_from_penalty', split(df11['team'], ' ').getItem(7)) \
       .withColumn('Punches_made', split(df11['team'], ' ').getItem(8)) \
       .withColumn('Match_played', split(df11['team'], ' ').getItem(9))

df11 = df11.drop(col("team"))
df11 = df11.na.drop()
df11 = df11.drop(col("Tiret"))
df11.show(truncate=False)

+----+----------+----------------+----------+----------+-------------+------------------+------------+------------+
|Rank|Nom       |Equipe          |Poste     |Save      |Goal_conceded|Saves_from_penalty|Punches_made|Match_played|
+----+----------+----------------+----------+----------+-------------+------------------+------------+------------+
|01  |Trubin    |Shakhtar_Donetsk|Goalkeeper|31        |10           |0                 |10          |6           |
|01  |Mignolet  |Club_Brugge     |Goalkeeper|31        |4            |0                 |5           |6           |
|03  |Onana     |Inter           |Goalkeeper|29        |7            |0                 |4           |6           |
|04  |Livaković |Dinamo_Zagreb   |Goalkeeper|25        |11           |0                 |2           |6           |
|05  |Staněk    |Plzeň           |Goalkeeper|24        |19           |0                 |2           |5           |
|05  |McGregor  |Rangers         |Goalkeeper|24        |18           |1 

In [51]:
df11.toPandas().to_csv("./data/LDC/ucl_players_gk.csv")

### Players Attack

In [52]:
path = "./data/LDC/ucl_players_attack.csv"
df12 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [53]:
df12 = df12.drop(col("_c0"))
df12 = df12.withColumn('Rank', split(df12['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df12['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df12['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df12['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df12['team'], ' ').getItem(4)) \
       .withColumn('Assist', split(df12['team'], ' ').getItem(5)) \
       .withColumn('Corner_taken', split(df12['team'], ' ').getItem(6)) \
       .withColumn('Offside', split(df12['team'], ' ').getItem(7)) \
       .withColumn('Dribbles', split(df12['team'], ' ').getItem(9)) \
       .withColumn('Match_played', split(df12['team'], ' ').getItem(8))

df12 = df12.drop(col("team"))
df12 = df12.na.drop()
df12 = df12.drop(col("Tiret"))
df12.show(truncate=False)

+----+-------------+---------+----------+----------+------------+-------+--------+------------+
|Rank|Nom          |Equipe   |Poste     |Assist    |Corner_taken|Offside|Dribbles|Match_played|
+----+-------------+---------+----------+----------+------------+-------+--------+------------+
|01  |Messi        |Paris    |Forward   |4         |8           |0      |17      |5           |
|01  |João         |Cancelo  |-         |Defender  |4           |0      |6       |2           |
|01  |Diogo        |Jota     |-         |Forward   |4           |0      |4       |0           |
|04  |Kimmich      |Bayern   |Midfielder|3         |16          |0      |3       |5           |
|04  |Neymar       |Paris    |Forward   |3         |14          |0      |27      |5           |
|04  |Perišić      |Tottenham|Midfielder|3         |11          |1      |6       |5           |
|04  |De           |Bruyne   |-         |Midfielder|3           |9      |4       |0           |
|04  |Di           |María    |-         

In [54]:
df12.toPandas().to_csv("./data/LDC/ucl_players_attack.csv")

### Players Attempt

In [55]:
path = "./data/LDC/ucl_players_attempt.csv"
df13 = spark.read.option("delimiter",",").options(header=True).csv(path)


In [56]:
df13 = df13.drop(col("_c0"))
df13 = df13.withColumn('Rank', split(df13['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df13['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df13['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df13['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df13['team'], ' ').getItem(4)) \
       .withColumn('Total_attempts', split(df13['team'], ' ').getItem(5)) \
       .withColumn('Attempts_on_target', split(df13['team'], ' ').getItem(6)) \
       .withColumn('Attempts_off_target', split(df13['team'], ' ').getItem(7)) \
       .withColumn('Blocked', split(df13['team'], ' ').getItem(9)) \
       .withColumn('Match_played', split(df13['team'], ' ').getItem(8))

df13 = df13.drop(col("team"))
df13 = df13.na.drop()
df13 = df13.drop(col("Tiret"))
df13.show(truncate=False)

+----+-------------+-----------+-------+--------------+------------------+-------------------+-------+------------+
|Rank|Nom          |Equipe     |Poste  |Total_attempts|Attempts_on_target|Attempts_off_target|Blocked|Match_played|
+----+-------------+-----------+-------+--------------+------------------+-------------------+-------+------------+
|01  |Vinícius     |Júnior     |-      |Forward       |25                |12                 |6      |8           |
|01  |Lewandowski  |Barcelona  |Forward|25            |11                |6                  |8      |5           |
|03  |Kvaratskhelia|Napoli     |Forward|22            |6                 |10                 |6      |5           |
|04  |Mbappé       |Paris      |Forward|21            |13                |7                  |1      |6           |
|04  |Griezmann    |Atlético   |Forward|21            |11                |8                  |2      |6           |
|04  |Rafa         |Silva      |-      |Forward       |21               

In [57]:
df13.toPandas().to_csv("./data/LDC/ucl_players_attempt.csv")

### Players Goals

In [58]:
path = "./data/LDC/ucl_players_goals.csv"
df14 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [59]:
df14 = df14.drop(col("_c0"))
df14 = df14.withColumn('Rank', split(df14['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df14['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df14['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df14['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df14['team'], ' ').getItem(4)) \
       .withColumn('Goals', split(df14['team'], ' ').getItem(5)) \
       .withColumn('Right_foot', split(df14['team'], ' ').getItem(6)) \
       .withColumn('Left_foot', split(df14['team'], ' ').getItem(7)) \
       .withColumn('Head', split(df14['team'], ' ').getItem(13)) \
       .withColumn('Others', split(df14['team'], ' ').getItem(8)) \
       .withColumn('Goals_inside_area', split(df14['team'], ' ').getItem(9)) \
       .withColumn('Goals_outside', split(df14['team'], ' ').getItem(10)) \
       .withColumn('Penalty', split(df14['team'], ' ').getItem(11)) \
       .withColumn('Match_played', split(df14['team'], ' ').getItem(12))

df14 = df14.drop(col("team"))
df14 = df14.na.drop()
df14 = df14.drop(col("Tiret"))
df14.show(truncate=False)

+----+-------------+-----------+----------+----------+----------+---------+----+------+-----------------+-------------+-------+------------+
|Rank|Nom          |Equipe     |Poste     |Goals     |Right_foot|Left_foot|Head|Others|Goals_inside_area|Goals_outside|Penalty|Match_played|
+----+-------------+-----------+----------+----------+----------+---------+----+------+-----------------+-------------+-------+------------+
|01  |Mbappé       |Paris      |Forward   |7         |7         |0        |0   |0     |6                |1            |1      |6           |
|01  |Salah        |Liverpool  |Forward   |7         |0         |7        |0   |0     |7                |0            |1      |6           |
|03  |Taremi       |Porto      |Forward   |5         |5         |0        |0   |0     |4                |1            |2      |5           |
|03  |Lewandowski  |Barcelona  |Forward   |5         |2         |1        |2   |0     |4                |1            |0      |5           |
|03  |Haaland

In [60]:
df14.toPandas().to_csv("./data/LDC/ucl_players_goals.csv")

### Players Def

In [61]:
path = "./data/LDC/ucl_players_def.csv"
df15 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [62]:
df15 = df15.drop(col("_c0"))
df15 = df15.withColumn('Rank', split(df15['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df15['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df15['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df15['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df15['team'], ' ').getItem(4)) \
       .withColumn('Ball_recover', split(df15['team'], ' ').getItem(5)) \
       .withColumn('Tackles', split(df15['team'], ' ').getItem(6)) \
       .withColumn('Tackles_Won', split(df15['team'], ' ').getItem(7)) \
       .withColumn('Tackles_Lost', split(df15['team'], ' ').getItem(10)) \
       .withColumn('Clearance_Attempts', split(df15['team'], ' ').getItem(8)) \
       .withColumn('Match_played', split(df15['team'], ' ').getItem(9))

df15 = df15.drop(col("team"))
df15 = df15.na.drop()
df15 = df15.drop(col("Tiret"))
df15.show(truncate=False)

+----+----------+----------------+----------+------------+-------+-----------+------------+------------------+------------+
|Rank|Nom       |Equipe          |Poste     |Ball_recover|Tackles|Tackles_Won|Tackles_Lost|Clearance_Attempts|Match_played|
+----+----------+----------------+----------+------------+-------+-----------+------------+------------------+------------+
|01  |Otamendi  |Benfica         |Defender  |46          |12     |7          |5           |30                |6           |
|01  |Balerdi   |Marseille       |Defender  |46          |3      |1          |2           |17                |6           |
|03  |Hincapie  |Leverkusen      |Defender  |45          |11     |5          |6           |15                |5           |
|03  |Mechele   |Club_Brugge     |Defender  |45          |7      |4          |3           |29                |6           |
|05  |Perić     |Dinamo_Zagreb   |Defender  |43          |9      |7          |2           |41                |6           |
|05  |Šu

In [63]:
df15.toPandas().to_csv("./data/LDC/ucl_players_def.csv")

### Players Discip

In [64]:
path = "./data/LDC/ucl_players_discip.csv"
df16 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [65]:
df16 = df16.drop(col("_c0"))
df16 = df16.withColumn('Rank', split(df16['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df16['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df16['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df16['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df16['team'], ' ').getItem(4)) \
       .withColumn('Fouls_Commited', split(df16['team'], ' ').getItem(5)) \
       .withColumn('Fouls_Suffered', split(df16['team'], ' ').getItem(6)) \
       .withColumn('Yellow_card', split(df16['team'], ' ').getItem(7)) \
       .withColumn('Red_card', split(df16['team'], ' ').getItem(10)) \
       .withColumn('Minutes_played', split(df16['team'], ' ').getItem(8)) \
       .withColumn('Match_played', split(df16['team'], ' ').getItem(9))

df16 = df16.drop(col("team"))
df16 = df16.na.drop()
df16 = df16.drop(col("Tiret"))
df16.show(truncate=False)

+----+---------+-------------+----------+--------------+--------------+-----------+--------+--------------+------------+
|Rank|Nom      |Equipe       |Poste     |Fouls_Commited|Fouls_Suffered|Yellow_card|Red_card|Minutes_played|Match_played|
+----+---------+-------------+----------+--------------+--------------+-----------+--------+--------------+------------+
|01  |Ugarte   |Sporting     |-         |Midfielder    |16            |10         |6       |2             |468         |
|01  |Fernandez|Benfica      |Midfielder|16            |4             |3          |0       |429           |5           |
|03  |Andrich  |Leverkusen   |Midfielder|15            |2             |4          |0       |409           |5           |
|04  |Sterling |Chelsea      |Midfielder|14            |5             |1          |0       |467           |6           |
|04  |Rongier  |Marseille    |Midfielder|14            |4             |0          |0       |471           |6           |
|06  |Abu      |Fani         |- 

In [66]:
df16.toPandas().to_csv("./data/LDC/ucl_players_discip.csv")

### Players Distrib

In [67]:
path = "./data/LDC/ucl_players_distrib.csv"
df17 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [68]:
df17 = df17.drop(col("_c0"))
df17 = df17.withColumn('Rank', split(df17['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df17['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df17['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df17['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df17['team'], ' ').getItem(4)) \
       .withColumn('Pass_accuracy', split(df17['team'], ' ').getItem(5)) \
       .withColumn('Pass_attempted', split(df17['team'], ' ').getItem(6)) \
       .withColumn('Pass_completed', split(df17['team'], ' ').getItem(7)) \
       .withColumn('Cross_accuracy', split(df17['team'], ' ').getItem(12)) \
       .withColumn('Cross_attempted', split(df17['team'], ' ').getItem(9)) \
       .withColumn('Cross_completed', split(df17['team'], ' ').getItem(8)) \
       .withColumn('Freekick', split(df17['team'], ' ').getItem(10)) \
       .withColumn('Match_played', split(df17['team'], ' ').getItem(11))

df17 = df17.drop(col("team"))
df17 = df17.na.drop()
df17 = df17.drop(col("Tiret"))
df17.show(truncate=False)

+----+--------------+-----------+----------+-------------+--------------+--------------+--------------+---------------+---------------+--------+------------+
|Rank|Nom           |Equipe     |Poste     |Pass_accuracy|Pass_attempted|Pass_completed|Cross_accuracy|Cross_attempted|Cross_completed|Freekick|Match_played|
+----+--------------+-----------+----------+-------------+--------------+--------------+--------------+---------------+---------------+--------+------------+
|01  |Kampl         |Leipzig    |Midfielder|98.8         |150           |146           |0             |0              |0              |3       |4           |
|02  |Eric          |García     |-         |Defender     |98.5          |184           |4             |1              |180            |1       |1           |
|03  |Aké           |Man_City   |Defender  |98           |247           |239           |0             |0              |0              |1       |3           |
|03  |A.            |Christensen|-         |Defender

In [69]:
df17.toPandas().to_csv("./data/LDC/ucl_players_distrib.csv")

### Players Key

In [70]:
path = "./data/LDC/ucl_players_key.csv"
df18 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [71]:
df18 = df18.drop(col("_c0"))
df18 = df18.withColumn('Rank', split(df18['team'], ' ').getItem(0)) \
       .withColumn('Nom', split(df18['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df18['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df18['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df18['team'], ' ').getItem(4)) \
       .withColumn('Minute_played', split(df18['team'], ' ').getItem(5)) \
       .withColumn('Match_played', split(df18['team'], ' ').getItem(6)) \
       .withColumn('Goals', split(df18['team'], ' ').getItem(7)) \
       .withColumn('Assist', split(df18['team'], ' ').getItem(8)) \
       .withColumn('Distance_covered', split(df18['team'], ' ').getItem(9)) \
       .withColumn('Top_speed', split(df18['team'], ' ').getItem(10)) 

df18 = df18.drop(col("team"))
df18 = df18.drop(col("Tiret"))
df18 = df18.na.drop()
df18.show(truncate=False)

+----+----------+----------------+----------+-------------+------------+-----+------+----------------+---------+
|Rank|Nom       |Equipe          |Poste     |Minute_played|Match_played|Goals|Assist|Distance_covered|Top_speed|
+----+----------+----------------+----------+-------------+------------+-----+------+----------------+---------+
|01  |Grimaldo  |Benfica         |Defender  |540          |6           |2    |2     |67.2            |32.2     |
|01  |Chery     |M._Haifa        |Forward   |540          |6           |2    |1     |67.6            |30.3     |
|01  |Kane      |Tottenham       |Forward   |540          |6           |1    |2     |66.8            |31.5     |
|01  |Seiwald   |Salzburg        |Midfielder|540          |6           |1    |0     |70.3            |34.1     |
|01  |Ljubičić  |Dinamo_Zagreb   |Midfielder|540          |6           |1    |0     |63.9            |31.7     |
|01  |N'Dicka   |Frankfurt       |Defender  |540          |6           |0    |1     |66.8       

In [72]:
df18.toPandas().to_csv("./data/LDC/ucl_players_key.csv")

## EL

### Games 

In [126]:
path = "./data/EL/el_games.csv"
df19 = spark.read.option("delimiter",",").options(header=True).csv(path)
df19 = df19.drop(col("_c0"))
df19.show()

+------------+------------------+---------+---------+------------------+
|        date|        equipe_dom|score_dom|score_ext|        equipe_ext|
+------------+------------------+---------+---------+------------------+
|03.11. 21:00|           Arsenal|        1|        0|            Zurich|
|03.11. 21:00|           AS Roma|        3|        1|        Ludogorets|
|03.11. 21:00|             Betis|        3|        0|               HJK|
|03.11. 21:00|        Bodo/Glimt|        1|        2|               PSV|
|03.11. 21:00|             Braga|        2|        1|          Malmo FF|
|03.11. 21:00|         Dyn. Kyiv|        0|        2|        Fenerbahce|
|03.11. 21:00|            Rennes|        1|        1|       AEK Larnaca|
|03.11. 21:00|   Royale Union SG|        0|        1|      Union Berlin|
|03.11. 18:45|         Feyenoord|        1|        0|             Lazio|
|03.11. 18:45|       Midtjylland|        2|        0|        Sturm Graz|
|03.11. 18:45|            Monaco|        4|        

In [127]:
df19.toPandas().to_csv("./data/EL/el_games.csv")

### Group

In [182]:
path = "./data/EL/el_group.csv"
df20 = spark.read.option("delimiter",",").options(header=True).csv(path)
df20 = df20.drop(col("_c0"))
df20.show(32)

+-------------+--------+---+----+----+---+---+---+---+
|         team|nb_match|win|draw|lose| GF| GA| GD|pts|
+-------------+--------+---+----+----+---+---+---+---+
|      Arsenal|       6|  5|   0|   1|  8|  3|  5| 15|
|          PSV|       6|  4|   1|   1| 15|  4| 11| 13|
|   Bodø/Glimt|       6|  1|   1|   4|  5| 10| -5|  4|
|       Zürich|       6|  1|   0|   5|  5| 16|-11|  3|
|   Fenerbahçe|       6|  4|   2|   0| 13|  7|  6| 14|
|       Rennes|       6|  3|   3|   0| 11|  8|  3| 12|
|  AEK Larnaca|       6|  1|   2|   3|  7| 10| -3|  5|
|  Dynamo Kyiv|       6|  0|   1|   5|  5| 11| -6|  1|
|        Betis|       6|  5|   1|   0| 12|  4|  8| 16|
|         Roma|       6|  3|   1|   2| 11|  7|  4| 10|
|   Ludogorets|       6|  2|   1|   3|  8|  9| -1|  7|
|          HJK|       6|  0|   1|   5|  2| 13|-11|  1|
|     Union SG|       6|  4|   1|   1| 11|  7|  4| 13|
| Union Berlin|       6|  4|   0|   2|  4|  2|  2| 12|
|        Braga|       6|  3|   1|   2|  9|  7|  2| 10|
|        M

In [184]:
from pyspark.sql.functions import when
df20 = df20.withColumn("Group", \
   when((df20.team == "Arsenal"), lit("A")) \
     .when((df20.team == "PSV"), lit("A")) \
     .when((df20.team == "Bodø/Glimt") , lit("A")) \
     .when((df20.team == "Zürich") , lit("A")) \
     .when((df20.team == "Fenerbahçe"), lit("B")) \
     .when((df20.team == "Rennes") , lit("B")) \
     .when((df20.team == "AEK Larnaca") , lit("B")) \
     .when((df20.team == "Dynamo Kyiv") , lit("B")) \
     .when((df20.team == "Betis"), lit("C")) \
     .when((df20.team == "Roma") , lit("C")) \
     .when((df20.team == "Ludogorets") , lit("C")) \
     .when((df20.team == "HJK") , lit("C")) \
     .when((df20.team == "Union SG") , lit("D")) \
     .when((df20.team == "Union Berlin") , lit("D")) \
     .when((df20.team == "Braga") , lit("D")) \
     .when((df20.team == "Malmö") , lit("D")) \
     .when((df20.team == "Real Sociedad") , lit("E")) \
     .when((df20.team == "Man United") , lit("E")) \
     .when((df20.team == "Sheriff") , lit("E")) \
     .when((df20.team == "Omonoia") , lit("E")) \
     .when((df20.team == "Feyenoord") , lit("F")) \
     .when((df20.team == "Midtjylland") , lit("F")) \
     .when((df20.team == "Lazio") , lit("F")) \
     .when((df20.team == "Sturm") , lit("F")) \
     .when((df20.team == "Freiburg") , lit("G")) \
     .when((df20.team == "Nantes") , lit("G")) \
     .when((df20.team == "Qarabağ") , lit("G")) \
     .when((df20.team == "Olympiacos") , lit("G")) \
     .when((df20.team == "Ferencváros") , lit("H")) \
     .when((df20.team == "Monaco") , lit("H")) \
     .when((df20.team == "Trabzonspor") , lit("H")) \
     .when((df20.team == "Crvena zvezda" ) , lit("H")))
df20.show(32)     

+-------------+--------+---+----+----+---+---+---+---+-----+
|         team|nb_match|win|draw|lose| GF| GA| GD|pts|Group|
+-------------+--------+---+----+----+---+---+---+---+-----+
|      Arsenal|       6|  5|   0|   1|  8|  3|  5| 15|    A|
|          PSV|       6|  4|   1|   1| 15|  4| 11| 13|    A|
|   Bodø/Glimt|       6|  1|   1|   4|  5| 10| -5|  4|    A|
|       Zürich|       6|  1|   0|   5|  5| 16|-11|  3|    A|
|   Fenerbahçe|       6|  4|   2|   0| 13|  7|  6| 14|    B|
|       Rennes|       6|  3|   3|   0| 11|  8|  3| 12|    B|
|  AEK Larnaca|       6|  1|   2|   3|  7| 10| -3|  5|    B|
|  Dynamo Kyiv|       6|  0|   1|   5|  5| 11| -6|  1|    B|
|        Betis|       6|  5|   1|   0| 12|  4|  8| 16|    C|
|         Roma|       6|  3|   1|   2| 11|  7|  4| 10|    C|
|   Ludogorets|       6|  2|   1|   3|  8|  9| -1|  7|    C|
|          HJK|       6|  0|   1|   5|  2| 13|-11|  1|    C|
|     Union SG|       6|  4|   1|   1| 11|  7|  4| 13|    D|
| Union Berlin|       6|

In [185]:
df20.toPandas().to_csv("./data/EL/el_group.csv")

### Attack 

In [130]:
path = "./data/EL/el_stats_clubs_attack.csv"
df21 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [131]:
df21 = df21.drop(col("_c0"))
df21 = df21.withColumn('Rank', split(df21['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df21['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df21['team'], ' ').getItem(2)) \
       .withColumn('Attack', split(df21['team'], ' ').getItem(3)) \
       .withColumn('Assist', split(df21['team'], ' ').getItem(4)) \
       .withColumn('Corner_taken', split(df21['team'], ' ').getItem(5)) \
       .withColumn('Offside', split(df21['team'], ' ').getItem(6)) \
       .withColumn('Match_played', split(df21['team'], ' ').getItem(7))

df21 = df21.drop(col("team"))
df21 = df21.na.drop()
df21.show(truncate=False)

AnalysisException: Cannot resolve column name "team" among (Rank, Equipe, Country, Attack, Assist, Corner_taken, Offside, Match_played)

In [75]:
df21.toPandas().to_csv("./data/EL/el_stats_clubs_attack.csv")

### Players Attack 

In [141]:
path = "./data/EL/el_stats_player_attack.csv"
df22 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [142]:
df22 = df22.drop(col("_c0"))
df22 = df22.withColumn('Rank', split(df22['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df22['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df22['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df22['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df22['team'], ' ').getItem(4)) \
       .withColumn('Assist', split(df22['team'], ' ').getItem(5)) \
       .withColumn('Corner_taken', split(df22['team'], ' ').getItem(6)) \
       .withColumn('Offside', split(df22['team'], ' ').getItem(7)) \
       .withColumn('Match_played', split(df22['team'], ' ').getItem(8))

df22 = df22.drop(col("team"))
df22 = df22.na.drop()
df22 = df22.drop(col("Tiret"))
df22.show(truncate=False)

+----+------------+-----------+----------+------+------------+-------+------------+
|Rank|Nom         |Equipe     |Poste     |Assist|Corner_taken|Offside|Match_played|
+----+------------+-----------+----------+------+------------+-------+------------+
|01  |Evander     |Midtjylland|Midfielder|5     |12          |1      |6           |
|02  |Bakasetas   |Trabzonspor|Midfielder|4     |17          |0      |6           |
|03  |Pellegrini  |Roma       |Midfielder|3     |29          |1      |5           |
|03  |Gakpo       |PSV        |Forward   |3     |10          |5      |5           |
|03  |Rossi       |Fenerbahçe |Midfielder|3     |10          |1      |4           |
|03  |Blas        |Nantes     |Midfielder|3     |5           |0      |6           |
|03  |Miranda     |Betis      |Defender  |3     |3           |0      |6           |
|08  |Kady        |Qarabağ    |Midfielder|2     |29          |2      |6           |
|08  |Bourigeaud  |Rennes     |Midfielder|2     |20          |1      |6     

In [143]:
df22.toPandas().to_csv("./data/EL/el_stats_players_attack.csv")

### Attempt


In [121]:
path = "./data/EL/el_stats_clubs_attempts.csv"
df23 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [122]:
df23 = df23.drop(col("_c0"))
df23 = df23.withColumn('Rank', split(df23['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df23['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df23['team'], ' ').getItem(2)) \
       .withColumn('Total_attempt', split(df23['team'], ' ').getItem(3)) \
       .withColumn('Attempt_on_target', split(df23['team'], ' ').getItem(4)) \
       .withColumn('Attempt_off_target', split(df23['team'], ' ').getItem(5)) \
       .withColumn('Blocked', split(df23['team'], ' ').getItem(6)) \
       .withColumn('Match_played', split(df23['team'], ' ').getItem(7)) 

df23 = df23.drop(col("team"))
df23 = df23.na.drop()
df23.show(truncate=False)

AnalysisException: Cannot resolve column name "team" among (Rank, Equipe, Country, Total_attempt, Attempt_on_target, Attempt_off_target, Blocked, Match_played)

In [123]:
df23.toPandas().to_csv("./data/EL/el_stats_clubs_attempts.csv")

### Players Attempt


In [124]:
path = "./data/EL/el_stats_players_attempts.csv"
df24 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [125]:
df24 = df24.drop(col("_c0"))
df24 = df24.withColumn('Rank', split(df24['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df24['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df24['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df24['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df24['team'], ' ').getItem(4)) \
       .withColumn('Total_attempts', split(df24['team'], ' ').getItem(5)) \
       .withColumn('Attempts_on_target', split(df24['team'], ' ').getItem(6)) \
       .withColumn('Attempts_off_target', split(df24['team'], ' ').getItem(7)) \
       .withColumn('Blocked', split(df24['team'], ' ').getItem(8)) \
       .withColumn('Match_played', split(df24['team'], ' ').getItem(9))

df24 = df24.drop(col("team"))
df24 = df24.na.drop()
df24 = df24.drop(col("Tiret"))
df24.show(truncate=False)

AnalysisException: Cannot resolve column name "team" among (Rank, Nom, Equipe, Poste, Total_attempts, Attempts_on_target, Attempts_off_target, Blocked, Match_played)

In [84]:
df24.toPandas().to_csv("./data/EL/el_stats_players_attempts.csv")

### Def

In [85]:
path = "./data/EL/el_stats_clubs_defending.csv"
df25 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [86]:
df25 = df25.drop(col("_c0"))
df25 = df25.withColumn('Rank', split(df25['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df25['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df25['team'], ' ').getItem(2)) \
       .withColumn('Ball_recover', split(df25['team'], ' ').getItem(3)) \
       .withColumn('Tackles', split(df25['team'], ' ').getItem(4)) \
       .withColumn('Tackles_Won', split(df25['team'], ' ').getItem(5)) \
       .withColumn('Tackles_Lost', split(df25['team'], ' ').getItem(6)) \
       .withColumn('Clearence_attempt', split(df25['team'], ' ').getItem(7)) \
       .withColumn('Match_played', split(df25['team'], ' ').getItem(8))

df25 = df25.drop(col("team"))
df25 = df25.na.drop()
df25.show(truncate=False)

+----+-------------+-----------+------------+-------+-----------+------------+-----------------+------------+
|Rank|Equipe       |Country    |Ball_recover|Tackles|Tackles_Won|Tackles_Lost|Clearence_attempt|Match_played|
+----+-------------+-----------+------------+-------+-----------+------------+-----------------+------------+
|01  |Monaco       |France     |261         |55     |-          |-           |54               |6           |
|02  |PSV          |Netherlands|253         |69     |-          |-           |90               |6           |
|02  |Crvena_zvezda|Serbia     |253         |48     |-          |-           |98               |6           |
|04  |Lazio        |Italy      |251         |50     |-          |-           |66               |6           |
|05  |Nantes       |France     |246         |55     |-          |-           |119              |6           |
|06  |Trabzonspor  |Türkiye    |244         |47     |-          |-           |118              |6           |
|07  |Feye

In [87]:
df25.toPandas().to_csv("./data/EL/el_stats_clubs_defending.csv")

### Players Def

In [88]:
path = "./data/EL/el_stats_players_defending.csv"
df26 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [89]:
df26 = df26.drop(col("_c0"))
df26 = df26.withColumn('Rank', split(df26['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df26['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df26['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df26['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df26['team'], ' ').getItem(4)) \
       .withColumn('Ball_recover', split(df26['team'], ' ').getItem(5)) \
       .withColumn('Tackles', split(df26['team'], ' ').getItem(6)) \
       .withColumn('Tackles_Won', split(df26['team'], ' ').getItem(7)) \
       .withColumn('Tackles_Lost', split(df26['team'], ' ').getItem(8)) \
       .withColumn('Clearance_Attempts', split(df26['team'], ' ').getItem(9)) \
       .withColumn('Match_played', split(df26['team'], ' ').getItem(10))

df26 = df26.drop(col("team"))
df26 = df26.na.drop()
df26 = df26.drop(col("Tiret"))
df26.show(truncate=False)

+----+-----------+-------------+--------+------------+-------+-----------+------------+------------------+------------+
|Rank|Nom        |Equipe       |Poste   |Ball_recover|Tackles|Tackles_Won|Tackles_Lost|Clearance_Attempts|Match_played|
+----+-----------+-------------+--------+------------+-------+-----------+------------+------------------+------------+
|01  |Burgess    |Union_SG     |Defender|53          |6      |-          |-           |30                |6           |
|02  |Bartra     |Trabzonspor  |Defender|52          |9      |-          |-           |28                |6           |
|03  |Eraković   |Crvena_zvezda|Defender|49          |3      |-          |-           |14                |6           |
|04  |Mustafazade|Qarabağ      |Defender|43          |4      |-          |-           |33                |6           |
|04  |Disasi     |Monaco       |Defender|43          |4      |-          |-           |12                |6           |
|06  |Miličević  |AEK_Larnaca  |Defender

In [90]:
df26.toPandas().to_csv("./data/EL/el_stats_players_defending.csv")

### Discip

In [132]:
path = "./data/EL/el_stats_clubs_discip.csv"
df27 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [133]:
df27 = df27.drop(col("_c0"))
df27 = df27.withColumn('Rank', split(df27['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df27['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df27['team'], ' ').getItem(2)) \
       .withColumn('Fouls_Commited', split(df27['team'], ' ').getItem(3)) \
       .withColumn('Fouls_Suffered', split(df27['team'], ' ').getItem(4)) \
       .withColumn('Yellow_card', split(df27['team'], ' ').getItem(5)) \
       .withColumn('Red_card', split(df27['team'], ' ').getItem(6)) \
       .withColumn('Match_Played', split(df27['team'], ' ').getItem(7)) 


df27 = df27.drop(col("team"))
df27 = df27.na.drop()
df27.show(truncate=False)

+----+-------------+-----------+--------------+--------------+-----------+--------+------------+
|Rank|Equipe       |Country    |Fouls_Commited|Fouls_Suffered|Yellow_card|Red_card|Match_Played|
+----+-------------+-----------+--------------+--------------+-----------+--------+------------+
|01  |Fenerbahçe   |Türkiye    |99            |93            |17         |0       |6           |
|02  |Union_Berlin |Germany    |94            |63            |17         |2       |6           |
|03  |Dynamo_Kyiv  |Ukraine    |91            |99            |17         |1       |6           |
|04  |Crvena_zvezda|Serbia     |86            |78            |17         |1       |6           |
|04  |Rennes       |France     |86            |70            |11         |1       |6           |
|06  |AEK_Larnaca  |Cyprus     |84            |87            |19         |1       |6           |
|07  |Union_SG     |Belgium    |83            |74            |13         |0       |6           |
|08  |Real_Sociedad|Spain     

In [134]:
df27.toPandas().to_csv("./data/EL/el_stats_clubs_discip.csv")

### Players Discip

In [135]:
path = "./data/EL/el_stats_players_discip.csv"
df28 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [136]:
df28 = df28.drop(col("_c0"))
df28 = df28.withColumn('Rank', split(df28['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df28['team'], ' ').getItem(1)) \
        .withColumn('Equipe', split(df28['team'], ' ').getItem(2)) \
        .withColumn('Tiret', split(df28['team'], ' ').getItem(3)) \
        .withColumn('Poste', split(df28['team'], ' ').getItem(4)) \
        .withColumn('Fouls_Commited', split(df28['team'], ' ').getItem(5)) \
        .withColumn('Fouls_Suffered', split(df28['team'], ' ').getItem(6)) \
        .withColumn('Yellow_card', split(df28['team'], ' ').getItem(7)) \
        .withColumn('Red_card', split(df28['team'], ' ').getItem(8)) \
        .withColumn('Minutes_played', split(df28['team'], ' ').getItem(9)) \
        .withColumn('Match_played', split(df28['team'], ' ').getItem(10))

df28 = df28.drop(col("team"))
df28 = df28.na.drop()
df28 = df28.drop(col("Tiret"))
df28.show(truncate=False)

+----+---------------+-------------+----------+--------------+--------------+-----------+--------+--------------+------------+
|Rank|Nom            |Equipe       |Poste     |Fouls_Commited|Fouls_Suffered|Yellow_card|Red_card|Minutes_played|Match_played|
+----+---------------+-------------+----------+--------------+--------------+-----------+--------+--------------+------------+
|01  |Cicinho        |Ludogorets   |Defender  |16            |11            |3          |0       |447           |5           |
|02  |Sydorchuk      |Dynamo_Kyiv  |Midfielder|14            |11            |2          |0       |540           |6           |
|02  |Tekpetey       |Ludogorets   |Forward   |14            |11            |1          |0       |467           |6           |
|04  |Miguel_Crespo  |Fenerbahçe   |Midfielder|12            |13            |0          |0       |367           |5           |
|04  |Akouokou       |Betis        |Midfielder|12            |6             |2          |0       |358          

In [137]:
df28.toPandas().to_csv("./data/EL/el_stats_players_discip.csv")

### Distrib

In [119]:
path = "./data/EL/el_stats_clubs_distribution.csv"
df29 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [120]:
df29 = df29.drop(col("_c0"))
df29 = df29.withColumn('Rank', split(df29['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df29['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df29['team'], ' ').getItem(2)) \
       .withColumn('Pass_accuracy', split(df29['team'], ' ').getItem(3)) \
       .withColumn('Pass_attempted', split(df29['team'], ' ').getItem(4)) \
       .withColumn('Pass_completed', split(df29['team'], ' ').getItem(5)) \
       .withColumn('Cross_accuracy', split(df29['team'], ' ').getItem(7)) \
       .withColumn('Cross_attempted', split(df29['team'], ' ').getItem(8)) \
       .withColumn('Cross_completed', split(df29['team'], ' ').getItem(9)) \
       .withColumn('Freekick', split(df29['team'], ' ').getItem(10)) \
       .withColumn('Match_Played', split(df29['team'], ' ').getItem(11)) \
       .withColumn('Possesion', split(df29['team'], ' ').getItem(6)) 


df29 = df29.drop(col("team"))
df29 = df29.na.drop()
df29.show(truncate=False)

AnalysisException: Cannot resolve column name "team" among (Rank, Equipe, Country, Pass_accuracy, Pass_attempted, Pass_completed, Cross_accuracy, Cross_attempted, Cross_completed, Freekick, Match_Played, Possesion)

In [116]:
df29.toPandas().to_csv("./data/EL/el_stats_clubs_distribution.csv")

### Players Distrib

In [117]:
path = "./data/EL/el_stats_players_distribution.csv"
df30 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [118]:
df30 = df30.drop(col("_c0"))
df30 = df30.withColumn('Rank', split(df30['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df30['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df30['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df30['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df30['team'], ' ').getItem(4)) \
       .withColumn('Pass_accuracy', split(df30['team'], ' ').getItem(5)) \
       .withColumn('Pass_attempted', split(df30['team'], ' ').getItem(6)) \
       .withColumn('Pass_completed', split(df30['team'], ' ').getItem(7)) \
       .withColumn('Cross_accuracy', split(df30['team'], ' ').getItem(8)) \
       .withColumn('Cross_attempted', split(df30['team'], ' ').getItem(9)) \
       .withColumn('Cross_completed', split(df30['team'], ' ').getItem(10)) \
       .withColumn('Freekick', split(df30['team'], ' ').getItem(11)) \
       .withColumn('Match_played', split(df30['team'], ' ').getItem(12))

df30 = df30.drop(col("team"))
df30 = df30.na.drop()
df30 = df30.drop(col("Tiret"))
df30.show(truncate=False)

AnalysisException: Cannot resolve column name "team" among (Rank, Nom, Equipe, Poste, Pass_accuracy, Pass_attempted, Pass_completed, Cross_accuracy, Cross_attempted, Cross_completed, Freekick, Match_played)

In [113]:
df30.toPandas().to_csv("./data/EL/el_stats_players_distribution.csv")

### GK

In [108]:
path = "./data/EL/el_stats_clubs_gk.csv"
df31 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [109]:
df31 = df31.drop(col("_c0"))
df31 = df31.withColumn('Rank', split(df31['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df31['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df31['team'], ' ').getItem(2)) \
       .withColumn('Save', split(df31['team'], ' ').getItem(3)) \
       .withColumn('Goal_conceded', split(df31['team'], ' ').getItem(4)) \
       .withColumn('Own_goals_conceded', split(df31['team'], ' ').getItem(5)) \
       .withColumn('Saves_from_penalty', split(df31['team'], ' ').getItem(6)) \
       .withColumn('Punches_made', split(df31['team'], ' ').getItem(7)) \
       .withColumn('Match_played', split(df31['team'], ' ').getItem(8))

df31 = df31.drop(col("team"))
df31 = df31.na.drop()
df31.show(truncate=False)

+----+-------------+-----------+----+-------------+------------------+------------------+------------+------------+
|Rank|Equipe       |Country    |Save|Goal_conceded|Own_goals_conceded|Saves_from_penalty|Punches_made|Match_played|
+----+-------------+-----------+----+-------------+------------------+------------------+------------+------------+
|01  |Malmö        |Sweden     |33  |11           |0                 |0                 |0           |6           |
|02  |HJK          |Finland    |28  |13           |1                 |0                 |0           |6           |
|03  |Olympiacos   |Greece     |25  |11           |0                 |0                 |1           |6           |
|04  |Omonoia      |Cyprus     |24  |12           |0                 |0                 |0           |6           |
|04  |Ferencváros  |Hungary    |24  |9            |0                 |0                 |1           |6           |
|06  |Nantes       |France     |23  |11           |1                 |0 

In [110]:
df31.toPandas().to_csv("./data/EL/el_stats_clubs_gk.csv")

### Players GK

In [105]:
path = "./data/EL/el_stats_players_gk.csv"
df32 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [106]:
df32 = df32.drop(col("_c0"))
df32 = df32.withColumn('Rank', split(df32['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df32['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df32['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df32['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df32['team'], ' ').getItem(4)) \
       .withColumn('Save', split(df32['team'], ' ').getItem(5)) \
       .withColumn('Goal_conceded', split(df32['team'], ' ').getItem(6)) \
       .withColumn('Save_Penalty', split(df32['team'], ' ').getItem(7)) \
       .withColumn('Clean_sheet', split(df32['team'], ' ').getItem(8)) \
       .withColumn('Match_played', split(df32['team'], ' ').getItem(9))

df32 = df32.drop(col("team"))
df32 = df32.na.drop()
df32 = df32.drop(col("Tiret"))
df32.show(truncate=False)

+----+-------------+-------------+----------+----+-------------+------------+-----------+------------+
|Rank|Nom          |Equipe       |Poste     |Save|Goal_conceded|Save_Penalty|Clean_sheet|Match_played|
+----+-------------+-------------+----------+----+-------------+------------+-----------+------------+
|01  |Hazard       |HJK          |Goalkeeper|28  |13           |0           |0          |6           |
|02  |Dibusz       |Ferencváros  |Goalkeeper|24  |9            |0           |1          |6           |
|03  |Lafont       |Nantes       |Goalkeeper|23  |11           |0           |1          |6           |
|03  |Haikin       |Bodø/Glimt   |Goalkeeper|23  |10           |0           |0          |6           |
|05  |Pirić        |AEK_Larnaca  |Goalkeeper|22  |10           |1           |1          |6           |
|06  |Padt         |Ludogorets   |Goalkeeper|21  |9            |0           |1          |6           |
|06  |Magomedaliyev|Qarabağ      |Goalkeeper|21  |5            |0        

In [107]:
df32.toPandas().to_csv("./data/EL/el_stats_players_gk.csv")

### Goals

In [102]:
 path = "./data/EL/el_stats_clubs_goals.csv"
df33 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [103]:
df33 = df33.drop(col("_c0"))
df33 = df33.withColumn('Rank', split(df33['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df33['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df33['team'], ' ').getItem(2)) \
       .withColumn('Goals', split(df33['team'], ' ').getItem(3)) \
       .withColumn('Penalty', split(df33['team'], ' ').getItem(4)) \
       .withColumn('Match_Played', split(df33['team'], ' ').getItem(5)) 

df33= df33.drop(col("team"))
df33 = df33.na.drop()
df33.show(truncate=False)

+----+-------------+-----------+-----+-------+------------+
|Rank|Equipe       |Country    |Goals|Penalty|Match_Played|
+----+-------------+-----------+-----+-------+------------+
|01  |PSV          |Netherlands|15   |0      |6           |
|02  |Freiburg     |Germany    |13   |2      |6           |
|02  |Feyenoord    |Netherlands|13   |2      |6           |
|02  |Fenerbahçe   |Türkiye    |13   |2      |6           |
|05  |Midtjylland  |Denmark    |12   |1      |6           |
|05  |Betis        |Spain      |12   |1      |6           |
|07  |Roma         |Italy      |11   |3      |6           |
|07  |Union_SG     |Belgium    |11   |0      |6           |
|07  |Rennes       |France     |11   |0      |6           |
|07  |Trabzonspor  |Türkiye    |11   |0      |6           |
|11  |Man_United   |England    |10   |1      |6           |
|11  |Real_Sociedad|Spain      |10   |1      |6           |
|13  |Crvena_zvezda|Serbia     |9    |2      |6           |
|13  |Monaco       |France     |9    |2 

In [104]:
df33.toPandas().to_csv("./data/EL/el_stats_clubs_goals.csv")

### Players Goals

In [99]:
path = "./data/EL/el_stats_player_goals.csv"
df34 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [100]:
df34 = df34.drop(col("_c0"))
df34 = df34.withColumn('Rank', split(df34['team'], ' ').getItem(0)) \
        .withColumn('Nom', split(df34['team'], ' ').getItem(1)) \
       .withColumn('Equipe', split(df34['team'], ' ').getItem(2)) \
       .withColumn('Tiret', split(df34['team'], ' ').getItem(3)) \
       .withColumn('Poste', split(df34['team'], ' ').getItem(4)) \
       .withColumn('Goals', split(df34['team'], ' ').getItem(5)) \
       .withColumn('Penalty', split(df34['team'], ' ').getItem(6)) \
       .withColumn('Match_played', split(df34['team'], ' ').getItem(7))

df34 = df34.drop(col("team"))
df34 = df34.na.drop()
df34 = df34.drop(col("Tiret"))
df34.show(truncate=False)

+----+------------+-------------+----------+-----+-------+------------+
|Rank|Nom         |Equipe       |Poste     |Goals|Penalty|Match_played|
+----+------------+-------------+----------+-----+-------+------------+
|01  |Gimenez     |Feyenoord    |Forward   |4    |1      |6           |
|01  |Vitinha     |Braga        |Forward   |4    |0      |5           |
|03  |Kanga       |Crvena_zvezda|Midfielder|3    |2      |6           |
|03  |Pellegrini  |Roma         |Midfielder|3    |2      |5           |
|03  |Batshuayi   |Fenerbahçe   |Forward   |3    |1      |6           |
|03  |Ben_Yedder  |Monaco       |Forward   |3    |1      |6           |
|03  |Rick        |Ludogorets   |Forward   |3    |0      |6           |
|03  |Boniface    |Union_SG     |Forward   |3    |0      |6           |
|03  |Owusu       |Qarabağ      |Forward   |3    |0      |6           |
|03  |Zachariassen|Ferencváros  |Midfielder|3    |0      |6           |
|03  |Veerman     |PSV          |Midfielder|3    |0      |6     

In [101]:
df34.toPandas().to_csv("./data/EL/el_stats_player_goals.csv")

### Key

In [96]:
path = "./data/EL/el_stats_clubs_key.csv"
df35 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [97]:
df35 = df35.drop(col("_c0"))
df35 = df35.withColumn('Rank', split(df35['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df35['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df35['team'], ' ').getItem(2)) \
       .withColumn('Match_Played', split(df35['team'], ' ').getItem(3)) \
       .withColumn('Won', split(df35['team'], ' ').getItem(4)) \
       .withColumn('Drawn', split(df35['team'], ' ').getItem(5)) \
       .withColumn('Lost', split(df35['team'], ' ').getItem(6)) 

df35 = df35.drop(col("team"))
df35 = df35.na.drop()
df35.show(truncate=False)

+----+-------------+-----------+------------+---+-----+----+
|Rank|Equipe       |Country    |Match_Played|Won|Drawn|Lost|
+----+-------------+-----------+------------+---+-----+----+
|01  |Betis        |Spain      |6           |5  |1    |0   |
|01  |Man_United   |England    |6           |5  |0    |1   |
|01  |Arsenal      |England    |6           |5  |0    |1   |
|01  |Real_Sociedad|Spain      |6           |5  |0    |1   |
|01  |Freiburg     |Germany    |6           |4  |2    |0   |
|01  |Fenerbahçe   |Türkiye    |6           |4  |2    |0   |
|01  |Union_SG     |Belgium    |6           |4  |1    |1   |
|01  |PSV          |Netherlands|6           |4  |1    |1   |
|01  |Union_Berlin |Germany    |6           |4  |0    |2   |
|01  |Rennes       |France     |6           |3  |3    |0   |
|01  |Braga        |Portugal   |6           |3  |1    |2   |
|01  |Ferencváros  |Hungary    |6           |3  |1    |2   |
|01  |Roma         |Italy      |6           |3  |1    |2   |
|01  |Monaco       |Fran

In [98]:
df35.toPandas().to_csv("./data/EL/el_stats_clubs_key.csv")

### Players Key

In [93]:
path = "./data/EL/el_stats_player_key.csv"
df36 = spark.read.option("delimiter",",").options(header=True).csv(path)

In [94]:
df36 = df36.drop(col("_c0"))
df36 = df36.withColumn('Rank', split(df36['team'], ' ').getItem(0)) \
       .withColumn('Equipe', split(df36['team'], ' ').getItem(1)) \
       .withColumn('Country', split(df36['team'], ' ').getItem(2)) \
       .withColumn('Poste', split(df36['team'], ' ').getItem(4)) \
       .withColumn('Minutes_Played', split(df36['team'], ' ').getItem(5)) \
       .withColumn('Goals', split(df36['team'], ' ').getItem(6)) \
       .withColumn('Assist', split(df36['team'], ' ').getItem(7))\
       .withColumn('Match_played', split(df36['team'], ' ').getItem(8))


df36 = df36.drop(col("team"))
df36 = df36.na.drop()
df36.show(truncate=False)

+----+--------------+-------------+----------+--------------+-----+------+------------+
|Rank|Equipe        |Country      |Poste     |Minutes_Played|Goals|Assist|Match_played|
+----+--------------+-------------+----------+--------------+-----+------+------------+
|01  |Aitor_Ruibal  |Betis        |Forward   |540           |2    |0     |6           |
|01  |Hancko        |Feyenoord    |Defender  |540           |2    |0     |6           |
|01  |Disasi        |Monaco       |Defender  |540           |1    |1     |6           |
|01  |Lynen         |Union_SG     |Midfielder|540           |1    |0     |6           |
|01  |S._Mmaee      |Ferencváros  |Defender  |540           |1    |0     |6           |
|01  |Tomović       |AEK_Larnaca  |Defender  |540           |0    |0     |6           |
|01  |Verdon        |Ludogorets   |Defender  |540           |0    |0     |6           |
|01  |Haikin        |Bodø/Glimt   |Goalkeeper|540           |0    |0     |6           |
|01  |Eraković      |Crvena_zvez

In [95]:
df36.toPandas().to_csv("./data/EL/el_stats_player_key.csv")

## CDM

### Group

In [168]:
path = "./data/CDM/cdm_group.csv"
df37 = spark.read.option("delimiter",",").options(header=True).csv(path)
df37 = df37.drop(col("_c0"))

In [169]:
from pyspark.sql.functions import when
df37 = df37.withColumn("Group", \
   when((df37.standings == "Netherlands"), lit("A")) \
     .when((df37.standings == "Ecuador"), lit("A")) \
     .when((df37.standings == "Senegal") , lit("A")) \
     .when((df37.standings == "Qatar") , lit("A")) \
     .when((df37.standings == "England"), lit("B")) \
     .when((df37.standings == "USA") , lit("B")) \
     .when((df37.standings == "Iran") , lit("B")) \
     .when((df37.standings == "Wales") , lit("B")) \
     .when((df37.standings == "Poland"), lit("C")) \
     .when((df37.standings == "Argentina") , lit("C")) \
     .when((df37.standings == "Mexico") , lit("C")) \
     .when((df37.standings == "Saudi Arabia") , lit("C")) \
     .when((df37.standings == "France") , lit("D")) \
     .when((df37.standings == "Denmark")  , lit("D")) \
     .when((df37.standings == "Tunisia") , lit("D")) \
     .when((df37.standings == "Australia") , lit("D")) \
     .when((df37.standings == "Germany") , lit("E")) \
     .when((df37.standings == "Spain") , lit("E")) \
     .when((df37.standings == "Costa Rica") , lit("E")) \
     .when((df37.standings == "Japan") , lit("E")) \
     .when((df37.standings == "Croatia") , lit("F")) \
     .when((df37.standings == "Belgium") , lit("F")) \
     .when((df37.standings == "Canada") , lit("F")) \
     .when((df37.standings == "Morocco") , lit("F")) \
     .when((df37.standings == "Switzerland") , lit("G")) \
     .when((df37.standings == "Serbia") , lit("G")) \
     .when((df37.standings == "Brazil") , lit("G")) \
     .when((df37.standings == "Cameroon") , lit("G")) \
     .when((df37.standings == "Portugal") , lit("H")) \
     .when((df37.standings == "Uruguay") , lit("H")) \
     .when((df37.standings == "Ghana") , lit("H")) \
     .when((df37.standings == "South Korea" ) , lit("H")))
df37 = df37.drop(col("Groupe"))
df37.show(32)     

+------------+-----+
|   standings|Teams|
+------------+-----+
| Netherlands|    A|
|     Ecuador|    A|
|     Senegal|    A|
|       Qatar|    A|
|     England|    B|
|         USA|    B|
|        Iran|    B|
|       Wales|    B|
|      Poland|    C|
|   Argentina|    C|
|      Mexico|    C|
|Saudi Arabia|    C|
|      France|    D|
|     Denmark|    D|
|     Tunisia|    D|
|   Australia|    D|
|     Germany|    E|
|       Spain|    E|
|  Costa Rica|    E|
|       Japan|    E|
|     Croatia|    F|
|     Belgium|    F|
|      Canada|    F|
|     Morocco|    F|
| Switzerland|    G|
|      Serbia|    G|
|      Brazil|    G|
|    Cameroon|    G|
|    Portugal|    H|
|     Uruguay|    H|
|       Ghana|    H|
| South Korea|    H|
+------------+-----+



In [170]:
df37.toPandas().to_csv("./data/CDM/cdm_group.csv")

### Games

In [531]:
path = "./data/CDM/cdm_games.csv"
df38 = spark.read.option("delimiter",",").options(header=True).csv(path)
df38 = df38.drop(col("_c0"))
df38.show()

+------------+-----------+------------+
|        date| equipe_dom|  equipe_ext|
+------------+-----------+------------+
|20.11. 17:00|      Qatar|     Ecuador|
|21.11. 14:00|    England|        Iran|
|21.11. 17:00|    Senegal| Netherlands|
|21.11. 20:00|        USA|       Wales|
|22.11. 11:00|  Argentina|Saudi Arabia|
|22.11. 14:00|    Denmark|     Tunisia|
|22.11. 17:00|     Mexico|      Poland|
|22.11. 20:00|     France|   Australia|
|23.11. 11:00|    Morocco|     Croatia|
|23.11. 14:00|    Germany|       Japan|
|23.11. 17:00|      Spain|  Costa Rica|
|23.11. 20:00|    Belgium|      Canada|
|24.11. 11:00|Switzerland|    Cameroon|
|24.11. 14:00|    Uruguay| South Korea|
|24.11. 17:00|   Portugal|       Ghana|
|24.11. 20:00|     Brazil|      Serbia|
|25.11. 11:00|      Wales|        Iran|
|25.11. 14:00|      Qatar|     Senegal|
|25.11. 17:00|Netherlands|     Ecuador|
|25.11. 20:00|    England|         USA|
+------------+-----------+------------+
only showing top 20 rows



In [719]:
df38.toPandas().to_csv("./data/CDM/cdm_games.csv")